## First Movement: Periodic Defense with Poisson Attack Pattern

In [12]:
print("""This Cell is necessary or else my IDE will start running cells simultaneously which
      then leads to issues with unfinished imports""")

This Cell is necessary or else my IDE will start running cells simultaneously which
      then leads to issues with unfinished imports


In [13]:
import os
import logging
import numpy as np
from datetime import datetime
from scipy.stats import poisson

## New Stuff

In [14]:
# Check if variables exist, otherwise default to False
# image_mode = False

image_mode = globals().get('image_mode', False)
debug_mode = globals().get('debug_mode', False)
RUN_BASELINE_ONLY = globals().get('RUN_BASELINE_ONLY', False)

#image_mode = True
debug_mode = False
RUN_BASELINE_ONLY = False

print(f"Current settings: image_mode={image_mode}, debug_mode={debug_mode}, RUN_BASELINE_ONLY={RUN_BASELINE_ONLY}")

Current settings: image_mode=False, debug_mode=False, RUN_BASELINE_ONLY=False


In [15]:
experiment_name = "experiment_1"

In [16]:
# path construct
log_path = os.path.join(os.getcwd(), str(experiment_name) + '.log')

if os.path.exists(log_path):
    os.remove(log_path)

logger = logging.getLogger()
handler = logging.FileHandler(log_path, mode='w')
handler.setFormatter(logging.Formatter('%(message)s'))
logger.addHandler(handler)
logger.setLevel(logging.INFO)

# concat of name/date
logger.info('[1] "' + str(experiment_name) + '.log"')
logger.info('[1] "' + datetime.now().strftime("%a %b %d %H:%M:%S %Y") + '"')

# Set up subgraph logger 
if not RUN_BASELINE_ONLY: 
    subgraph_log_path = os.path.join(os.getcwd(), 'sub_' + str(experiment_name) + '.log')
    
    if os.path.exists(subgraph_log_path):
        os.remove(subgraph_log_path)
        
    subgraph_handler = logging.FileHandler(subgraph_log_path, mode='w')
    subgraph_handler.setFormatter(logging.Formatter('%(message)s'))
    

# Function to switch between loggers
def switch_logger(use_subgraph_logger=False):
    
    # remove handlers
    current_handlers = logger.handlers[:]
    i = 0
    while i < len(current_handlers):
        logger.removeHandler(current_handlers[i]) 
        i = i + 1
        
    if use_subgraph_logger:
        logger.addHandler(subgraph_handler)
        logger.info('[1] "sub_' + str(experiment_name) + '.log"')
        logger.info('[1] "' + datetime.now().strftime("%a %b %d %H:%M:%S %Y") + '"')
    else:
        logger.addHandler(handler)

In [17]:
# Global configuration
# This is the default weight value we will insert any time we we 
# need to insert a weight value for a trivial edge in the graph
# Why 0 ?  Because we have "hardness = exnp(-w)" so we get hardness = 1 for w = 0
# hardness = 1 means that edge is trivial to traverse
DEFAULT_WEIGHT_VALUE = 0  

In [18]:
%run attack_graph_MARA.ipynb

### Create Subgraphs

In [19]:
# if present it will use current_drop_percentage from extrapolate as drop percentage
# if not it will default to 20%
%run create_subgraphs.ipynb

In [ ]:
attack_rate_list = [4]  
defense_rate_list = [0]

def random_steps(route, attack_rate=None, defense_rate=None, graph=None): #random_steps
 
    if attack_rate is None:
        attack_rate = 2
    # Get Prob. Mass Function for values 0 to length-1
    pmf = poisson.pmf(np.arange(len(route)), attack_rate) #calc. the indiv. probabilities
    # Normalize 
    pmf = pmf / pmf.sum()
    return pmf

In [21]:
# %run ctr-core_tests.ipynb
%run exp_ctr-core.ipynb
main()

We start with the baseline graph calculation!
Now we are going to run the subgraph analysis
The subgraph analysis is done!


In [22]:
with open(experiment_name+'.log', 'r') as f:
    print(f.read())

[1] "experiment_1.log"
[1] "Tue Nov 25 12:42:42 2025"



BASELINE RUN: BOTH ATTACKER AND DEFENDER HAVE FULL GRAPH KNOWLEDGE



Debug - Current Attack paths:
  0: [1, 2, 3, 'c(6,9)']
  1: [1, 2, 4, 'c(6,9)']
  2: [1, 2, 5, 7, 8, 'c(6,9)']

+++++++++++++++++++++++++++++

The virtual target nodeid is c(6,9)

attack rate =  4 , defense rate =  0 

	equilibrium for multiobjective security game (MOSG)

optimal defense strategy:
         prob.
2 5.703806e-01
3 0.000000e+00
4 -0.000000e+00
5 0.000000e+00
7 0.000000e+00
8 4.296194e-01

worst case attack strategies per goal:
          1
1 0.3813949
2 0.3706134
3 0.2479917
[1] 0.177

Defender can keep attacker success below: 0.177
Attacker can guarantee success probability of: 0.177

